In [3]:
"/content/gDrive/MyDrive"

'/content/gDrive/MyDrive'

In [1]:
!pip install texthero -U
!pip install gensim
!pip install pyLDAvis==2.1.2
!pip install texthero
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 236 kB 7.4 MB/s 
     |████████████████████████████████| 10.4 MB 45.4 MB/s 
     |████████████████████████████████| 184 kB 40.8 MB/s 
     |████████████████████████████████| 1.0 MB 42.5 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.4
    Uninstalling srsly-2.4.4:
      Successfully uninstalled srsly-2.4.4
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.0
    Uninstalling thinc-8.1.0:
      Successfully uninstalled thinc-8.1.0
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.1
    Uninstalling spacy-3.4.1:
      Successfully uninstalled spacy-3.4.1
ERROR: pip's dependency resolver does not currently take into

In [11]:
# data manipulation
import pandas as pd
import numpy as np
import io 

import re
import nltk
from nltk.tokenize import TweetTokenizer
import string
import spacy


In [4]:
df=pd.read_csv('dataset.csv')

In [5]:
df.head()

,ID,AuthorID,Author,Date,Content,Words,"Change in appetite, losing or gaining weight",Sleeping too much or not sleeping well (insomnia),Fatigue and low energy most days,"Feeling worthless, guilty, and hopeless","An inability to focus and concentrate that may interfere with daily tasks at home, work, or school",Movements that are unusually slow or agitated (a change which is often noticeable to others),Thinking about death and dying; suicidal ideation or suicide attempts,None
0,1,4.560000e+17,Deleted User#0000,43630.77639,**List of International Suicide Hotlines**\n``...,72,N,N,N,N,N,N,N,Y
1,2,4.560000e+17,Deleted User#0000,43630.78611,"Hello, if you're depressed, being self-depreca...",56,N,N,N,Y,N,N,Y,N
2,3,4.680000e+17,angel?#5373,43630.84306,Please note that we are not professionals. Th...,61,N,N,N,N,N,N,N,Y
3,4,5.450000e+17,Sease#2410,43630.84514,yeah and people who just want attention don't ...,36,N,N,N,N,N,N,N,Y
4,5,3.030000e+17,Emma/Blue#2035,43630.85347,Just a small vent my arm hurts so much i know ...,41,N,N,N,Y,Y,Y,N,N


In [6]:
df=df.dropna()
df.isnull().sum()

ID                                                                                                    0
AuthorID                                                                                              0
Author                                                                                                0
Date                                                                                                  0
Content                                                                                               0
Words                                                                                                 0
Change in appetite, losing or gaining weight                                                          0
Sleeping too much or not sleeping well (insomnia)                                                     0
Fatigue and low energy most days                                                                      0
Feeling worthless, guilty, and hopeless                         

In [7]:
list(df)

['ID',
 'AuthorID',
 'Author',
 'Date',
 'Content',
 'Words',
 'Change in appetite, losing or gaining weight',
 'Sleeping too much or not sleeping well (insomnia)',
 'Fatigue and low energy most days',
 'Feeling worthless, guilty, and hopeless',
 'An inability to focus and concentrate that may interfere with daily tasks at home, work, or school',
 'Movements that are unusually slow or agitated (a change which is often noticeable to others)',
 'Thinking about death and dying; suicidal ideation or suicide attempts',
 'None']

In [8]:
df['None'] = df['None'].map({'Y': 1, 'N': 0})
df['Change in appetite, losing or gaining weight']=df['Change in appetite, losing or gaining weight'].map({'Y': 1, 'N': 0})
df['Sleeping too much or not sleeping well (insomnia)']=df['Sleeping too much or not sleeping well (insomnia)'].map({'Y': 1, 'N': 0})
df['Fatigue and low energy most days']=df['Fatigue and low energy most days'].map({'Y': 1, 'N': 0})
df['Feeling worthless, guilty, and hopeless']=df['Feeling worthless, guilty, and hopeless'].map({'Y': 1, 'N': 0})
df['An inability to focus and concentrate that may interfere with daily tasks at home, work, or school']=df['An inability to focus and concentrate that may interfere with daily tasks at home, work, or school'].map({'Y': 1, 'N': 0})
df['Movements that are unusually slow or agitated (a change which is often noticeable to others)']=df['Movements that are unusually slow or agitated (a change which is often noticeable to others)'].map({'Y': 1, 'N': 0})
df['Thinking about death and dying; suicidal ideation or suicide attempts']=df['Thinking about death and dying; suicidal ideation or suicide attempts'].map({'Y': 1, 'N': 0})

In [9]:
df.rename(columns={"Change in appetite, losing or gaining weight": "symp_one", "Sleeping too much or not sleeping well (insomnia)": "symp_two",
                  "Fatigue and low energy most days":'symp_three','Feeling worthless, guilty, and hopeless':'symp_four',
                  'An inability to focus and concentrate that may interfere with daily tasks at home, work, or school':'symp_five',
                  'Movements that are unusually slow or agitated (a change which is often noticeable to others)':'symp_six',
                  'Thinking about death and dying; suicidal ideation or suicide attempts':'symp_seven'},inplace=True)

In [10]:
df.head(5)

,ID,AuthorID,Author,Date,Content,Words,symp_one,symp_two,symp_three,symp_four,symp_five,symp_six,symp_seven,None
0,1,4.560000e+17,Deleted User#0000,43630.77639,**List of International Suicide Hotlines**\n``...,72,0,0,0,0.0,0,0,0.0,1.0
1,2,4.560000e+17,Deleted User#0000,43630.78611,"Hello, if you're depressed, being self-depreca...",56,0,0,0,1.0,0,0,1.0,0.0
2,3,4.680000e+17,angel?#5373,43630.84306,Please note that we are not professionals. Th...,61,0,0,0,0.0,0,0,0.0,1.0
3,4,5.450000e+17,Sease#2410,43630.84514,yeah and people who just want attention don't ...,36,0,0,0,0.0,0,0,0.0,1.0
4,5,3.030000e+17,Emma/Blue#2035,43630.85347,Just a small vent my arm hurts so much i know ...,41,0,0,0,1.0,1,1,0.0,0.0


### Cleaning Text Data

In [12]:
tt = TweetTokenizer()
tokens = df['Content'].apply(tt.tokenize)
print(tokens)

0        [*, *, List, of, International, Suicide, Hotli...
1        [Hello, ,, if, you're, depressed, ,, being, se...
2        [Please, note, that, we, are, not, professiona...
3        [yeah, and, people, who, just, want, attention...
4        [Just, a, small, vent, my, arm, hurts, so, muc...
                               ...                        
23989    [>, And, it, ‚, Äôs, just, getting, so, annoyi...
23990    [And, I, ‚, Äôm, mixed, so, I, get, called, th...
23991    [>, @SAD, BHABHIE, ., oof, that's, a, good, th...
23992    [>, And, I, ‚, Äôm, mixed, so, I, get, called,...
23993    [>, Yeah, I, ‚, Äôve, ignored, it, for, 5, yea...
Name: Content, Length: 23978, dtype: object


In [13]:
df['Content']=df['Content'].apply(lambda x: x.split(',,')[0])
df['Content']=df['Content'].apply(lambda x: x.split(',,,')[0])

In [14]:
# Dictionary of English Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

# Expanding Contractions in the reviews
df['Content']=df['Content'].apply(lambda x:expand_contractions(x))

In [15]:
#Lowering the comments
df['Content']=df['Content'].apply(lambda x: x.lower())

#Remove digits and words containing numbers
df['Content']=df['Content'].apply(lambda x: re.sub('\w*\d\w*','', x))

#Remove Punctuation
df['Content']=df['Content'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))

#Remove extra space
df['Content']=df['Content'].apply(lambda x: re.sub(' +',' ',x))


In [16]:
for index,text in enumerate(df['Content'][15:20]):
    print('data %d:\n'%(index+1),text)

data 1:
 i did that a few minutes ago but the thing is that it feels like a sharp pain
data 2:
 kinda got in a bike crash a couple of hours ago and hurt my ankle so there is my story 
data 3:
 okay if it is ankle then make sure the pain is coming from the scratch and not because you got a sprained ankle if it is the latter you need to place your leg on something higher than the rest of your body level to maintain proper blood flow fleekthumbsup and place some ice on it preferably 

if it is just a scratch and you are sure that is where the pain is coming from then clean it and put a bandage on and hope for the best lol
data 4:
 i was swimming and then i just got a really bad pain and could not walk without the pain
data 5:
 care üíï i hate my parents they treat me like cinderella they expect me to do everything like the dishes cleaning the counter tops etc when i was loading the dishwasher i called my nana a bitch because she was acting like one and my papa the abusive shit he is came

In [17]:
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

# Lemmatization with stopwords removal
df['Content']=df['Content'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

In [18]:
df.to_csv("cleaned_dataset.csv")